In [15]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models, preprocessing

# Загрузка файлов с локального компьютера

In [42]:
from google.colab import files
files.upload()

Saving letters.zip to letters.zip


{'letters.zip': b'PK\x03\x04\x14\x00\x00\x00\x00\x00\x9a)\x9eX\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00letters/PK\x03\x04\n\x00\x00\x00\x00\x00\x99\x96\x99Xu\x92\xfa\'8\n\x00\x008\n\x00\x00\x0f\x00\x00\x00letters/3.1.png\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00C\x00\x00\x00a\x08\x06\x00\x00\x00\x8d\xae\x1cN\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00\x00\x04gAMA\x00\x00\xb1\x8f\x0b\xfca\x05\x00\x00\x00\tpHYs\x00\x00\x12t\x00\x00\x12t\x01\xdef\x1fx\x00\x00\t\xcdIDATx^\xed\xdc\xd7\x8eSI\x17\x05\xe0\xa2\x81\x19`\xc8q\xc80\x0c"\xe7\x8cHB<\x01 \xf1.\\\xfd\x0f3\xdc1W\x08!\x82\x10\x02\x81D\x0e\x02\x81\xc89g\x9a\x1cf\xe6+\xa8\xfeO\x1bwcp\xfb84K*l\x1f\xfb\xd8g\xafZ{\xef\xda\xbbN\xd3\xe5\x9f\xff\x10\x8a\xe0\xe3\xc7\x8f\xe1\xfd\xfb\xf7\xe1\xed\xdb\xb7\xe1\xdd\xbbwq8\xd6\xc6\xc7k\x0e\xae\xb3K\x97.\xa1\xa9\xa9)t\xeb\xd6-t\xef\xde=\xfc\xfa\xeb\xaf\xe1\x97_~\t]\xbbv\x8d\xc3\xfbY|E\xc6\xa7O\x9f\xa2\xe1o\xde\xbc\t\xaf^\xbd\x8a\x8f\x8e\x81\x93\x0b\xbf\xa0\xd6Qx\xcd\x88AJ

In [44]:
!unzip letters.zip -d letters

Archive:  letters.zip
   creating: letters/letters/
 extracting: letters/letters/3.1.png  
  inflating: letters/letters/А.2.jpg  
  inflating: letters/letters/А.3.jpg  
  inflating: letters/letters/А2.jpg  
  inflating: letters/letters/А3.jpg  
  inflating: letters/letters/В.2.jpg  
 extracting: letters/letters/Д.1.png  
 extracting: letters/letters/Е.1.png  
  inflating: letters/letters/И.2.jpg  
  inflating: letters/letters/Л.2.jpg  
 extracting: letters/letters/О.1.png  
 extracting: letters/letters/П.1.png  
  inflating: letters/letters/Т.2.jpg  
  inflating: letters/letters/Ф.2.jpg  
  inflating: letters/letters/Ч.3.jpg  
  inflating: letters/letters/Щ.3.jpg  


# Загрузка датасета

In [3]:
import shutil
import os
import math

In [13]:
# Каталог с набором данных
data_dir = '/content/updating_dataset/updating_dataset'
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Часть набора данных для тестирования
test_data_portion = 0.15
# Часть набора данных для проверки
val_data_portion = 0.15

In [6]:
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    alphabet = "АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
    for letter in alphabet:
        os.makedirs(os.path.join(dir_name, letter))

In [7]:
create_directory(train_dir)
create_directory(val_dir)
create_directory(test_dir)

In [8]:
def copy_images(start_index, end_index, data_dir, dest_dir):
    characters = ["А", "Б", "В", "Г", "Д", "Е", "Ё", "Ж", "З", "И", "Й", "К", "Л", "М", "Н",
                  "О", "П", "Р", "С", "Т", "У", "Ф", "Х", "Ц", "Ч", "Ш", "Щ", "Ъ", "Ы", "Ь",
                  "Э", "Ю", "Я"]

    def count_items_in_directory(directory):
        items = os.listdir(directory)
        count = len(items)
        return count

    num_items = []

    for char in characters:
        directory_path = f"{data_dir}/{char}"
        num_items.append(count_items_in_directory(directory_path))

    for idx, char in enumerate(characters):
        numbers_images = num_items[idx]
        for i in range(math.floor(numbers_images * start_index), math.floor(numbers_images * end_index)):
            src_path = os.path.join(f"{data_dir}/{char}", f"{char}.{i+1}.jpg")
            dest_path = os.path.join(dest_dir, char)
            shutil.copy2(src_path, dest_path)

In [9]:
start_val_data_idx = 1 - val_data_portion - test_data_portion
start_test_data_idx = 1 - test_data_portion
end = 1

In [14]:
copy_images(0, start_val_data_idx, data_dir, train_dir)
copy_images(start_val_data_idx, start_test_data_idx, data_dir, val_dir)
copy_images(start_test_data_idx, end, data_dir, test_dir)
print("Копирование окончено")

Копирование окончено


# Создание архитектуры

In [33]:
mobilenet_v3_blocks = [
    (3, 16, 1, None),
    (3, 24, 2, None),
    (3, 24, 1, None),
    (5, 40, 2, 0.25),
    (5, 40, 1, 0.25),
    (5, 40, 1, 0.25),
    (3, 80, 2, None),
    (3, 80, 1, None),
    (3, 80, 1, None),
    (3, 112, 1, 0.25),
    (3, 112, 1, 0.25),
    (5, 160, 2, 0.25),
    (5, 160, 1, 0.25),
    (5, 160, 1, 0.25),
    (3, 240, 1, None),
    (3, 240, 1, None),
    (3, 240, 1, None),
    (3, 240, 1, None),
    (3, 240, 1, None)
]

def MobileNetV3(input_shape, num_classes):
    # Определение входных данных
    inputs = layers.Input(shape=input_shape)

    # Первый конволюционный слой
    x = layers.Conv2D(16, 3, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Блоки MobileNetV3
    for block_params in mobilenet_v3_blocks:
        kernel_size, num_filters, strides, se_ratio = block_params
        x = InvertedResidualBlock(x, num_filters, kernel_size, strides, se_ratio)

    # Глобальный пулинг
    x = layers.GlobalAveragePooling2D()(x)

    # Полносвязный классификационный слой
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)

def InvertedResidualBlock(inputs, num_filters, kernel_size, strides, se_ratio):
    # Сужение пути
    x = layers.Conv2D(num_filters // 2, 1, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Инверсный остаточный блок
    x = layers.DepthwiseConv2D(kernel_size, strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Squeeze and Excite блок
    if se_ratio is not None:
        x = SqueezeExcite(x, num_filters // 2, se_ratio)

    # Расширение пути
    x = layers.Conv2D(num_filters, 1, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Остаточное соединение
    if inputs.shape[-1] == num_filters:
        x = layers.Add()([inputs, x])

    return x

def SqueezeExcite(inputs, channels, ratio):
    # Squeeze операция
    x = layers.GlobalAveragePooling2D()(inputs)
    x = layers.Reshape((1, 1, channels))(x)

    # Excite операция
    x = layers.Conv2D(channels // ratio, 1, activation='relu', padding='same')(x)
    x = layers.Conv2D(channels, 1, activation='sigmoid', padding='same')(x)

    # Пересмотр каналов
    x = layers.Multiply()([inputs, x])

    return x

# Путь к каталогам с данными
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

# Предобработка данных
train_datagen = preprocessing.image.ImageDataGenerator(rescale=1./255)
val_datagen = preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(28,28),
    batch_size=32,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(28, 28),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(28,28),
    batch_size=32,
    class_mode='categorical')

Found 7920 images belonging to 33 classes.
Found 1716 images belonging to 33 classes.
Found 1716 images belonging to 33 classes.


In [34]:
# Построение модели MobileNetV3
model = MobileNetV3(input_shape=(224, 224, 3), num_classes=33)

In [35]:
# Компиляция модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
# Обучение модели
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator)


Epoch 1/10
248/248 [==============================] - 24s 98ms/step - loss: 0.3967 - accuracy: 0.8739 - val_loss: 0.6670 - val_accuracy: 0.7960
Epoch 2/10
248/248 [==============================] - 25s 100ms/step - loss: 0.3866 - accuracy: 0.8744 - val_loss: 0.5887 - val_accuracy: 0.8246
Epoch 3/10
248/248 [==============================] - 24s 98ms/step - loss: 0.3628 - accuracy: 0.8785 - val_loss: 0.6454 - val_accuracy: 0.8030
Epoch 4/10
248/248 [==============================] - 26s 105ms/step - loss: 0.3507 - accuracy: 0.8852 - val_loss: 0.9061 - val_accuracy: 0.7547
Epoch 5/10
248/248 [==============================] - 24s 99ms/step - loss: 0.3497 - accuracy: 0.8848 - val_loss: 1.0261 - val_accuracy: 0.7104
Epoch 6/10
248/248 [==============================] - 27s 108ms/step - loss: 0.3049 - accuracy: 0.8995 - val_loss: 0.6235 - val_accuracy: 0.8246
Epoch 7/10
248/248 [==============================] - 24s 97ms/step - loss: 0.3173 - accuracy: 0.9005 - val_loss: 1.2671 - val_accura

In [41]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc * 100:.2f}%')

54/54 [==============================] - 4s 75ms/step - loss: 0.6165 - accuracy: 0.8293
Test accuracy: 82.93%


In [58]:
class_names = ['Ё', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О',
               'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я']
blank = "/content/letters/letters/Л.2.jpg"
# blank_path = tf.keras.utils.get_file('blank', origin=blank)

img = tf.keras.utils.load_img(
    blank, target_size=(28,28)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

# make predictions
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
print(predictions)

# print inference result
print("На изображении скорее всего {} ({:.2f}% вероятность)".format(
	class_names[np.argmax(score)],
	100 * np.max(score)))

# show the image itself
img.show()

1/1 [==============================] - 0s 55ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0.]]
На изображении скорее всего Ш (7.83% вероятность)
